<h1 align='center'>Emotion Prediction

In [1]:
# Importing Libraries:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPooling2D
from keras.layers import LeakyReLU
import os

Using TensorFlow backend.


In [2]:
# Categories = Angry,Disgust,Fear,Happy,Neutral,Sad,Surprise
categories = 7
#Image size
img_rows,img_cols = 48,48
batch_size = 32

In [3]:
train_data_dir = r'D:\Facial-CNN\Images\train'
validation_data_dir = r'D:\Facial-CNN\Images\validation'

In [4]:
#Image Augmentation.
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=30,shear_range=0.3,zoom_range=0.3,width_shift_range=0.4,
                                   height_shift_range=0.4,horizontal_flip=True,fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(train_data_dir,color_mode='grayscale',target_size=(img_rows,img_cols),
                                                    batch_size=batch_size,class_mode='categorical',shuffle=True)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,color_mode='grayscale',
                                                              target_size=(img_rows,img_cols),batch_size=batch_size,
                                                              class_mode='categorical',shuffle=True)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [6]:
# Model Initialization.
model = Sequential()

In [7]:
#Input layer.
#Conv-1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='glorot_uniform',input_shape=(img_rows,img_cols,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [8]:
#Conv-2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [9]:
#Conv-3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [10]:
#Conv-4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

##### Fully connected feed forward network.

In [11]:
#The input matrix is flattened and provided as input to fully connected feedforward network.
#Layer 1
model.add(Flatten())
model.add(Dense(units=64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [12]:
#Layer 2
model.add(Dense(units=64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [13]:
#Layer 3 output-layer
model.add(Dense(categories,kernel_initializer='he_normal'))
model.add(Activation('softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)       

#### Adding Check poinst for early stopping.

In [14]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [15]:
checkpoint = ModelCheckpoint('Emotion_Model.h5',monitor='val_loss',mode='min',save_best_only=True,verbose=1)
#early stopping
earlystop = EarlyStopping(monitor='val_loss',min_delta=0,patience=3,verbose=1,restore_best_weights=True)
#reduced learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=3,verbose=1,min_delta=0.0001)

In [16]:
callbacks = [earlystop,checkpoint,reduce_lr]

##### specifying model optimizer and evaluation metrices

In [17]:
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.001),metrics=['accuracy'])

In [18]:
nb_train_samples = 28821
nb_validation_samples = 7066
epochs=20

#### model training

In [19]:
history=model.fit_generator(train_generator,steps_per_epoch=nb_train_samples//batch_size,epochs=epochs,callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/20
900/900 [==============================] - 417s 463ms/step - loss: 2.1656 - accuracy: 0.1948 - val_loss: 1.9162 - val_accuracy: 0.2605

Epoch 00001: val_loss improved from inf to 1.91625, saving model to Emotion_Model.h5
Epoch 2/20
900/900 [==============================] - 416s 462ms/step - loss: 1.8256 - accuracy: 0.2375 - val_loss: 2.0827 - val_accuracy: 0.1914

Epoch 00002: val_loss did not improve from 1.91625
Epoch 3/20
900/900 [==============================] - 405s 450ms/step - loss: 1.7950 - accuracy: 0.2518 - val_loss: 1.8553 - val_accuracy: 0.2821

Epoch 00003: val_loss improved from 1.91625 to 1.85531, saving model to Emotion_Model.h5
Epoch 4/20
900/900 [==============================] - 408s 454ms/step - loss: 1.7661 - accuracy: 0.2689 - val_loss: 1.7316 - val_accuracy: 0.2973

Epoch 00004: val_loss improved from 1.85531 to 1.73156, saving model to Emotion_Model.h5
Epoch 5/20
900/900 [==============================] - 405s 450ms/step - loss: 1.6986 - accuracy: 0

### Emotion prediction with trained model with Opencv haar cascade.

In [20]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

<b>OpenCV haarcascades:</b>
trained classifiers for detecting objects of a particular type, e.g. faces (frontal, profile), pedestrians etc.
https://github.com/opencv/opencv/tree/master/data/haarcascades

In [21]:
# loading haarcascade.
face_classifier = cv2.CascadeClassifier(r'D:\Facial-CNN\Scripts\haarcascade_frontalface_default.xml')
#loading trained model for prediction.
classifier =load_model(r'D:\Facial-CNN\Scripts\Emotion_Model.h5')
#types of class labels.
class_labels = ['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']

In [22]:
def PredictResult(isVideo=True,exp_frame=None):
    if isVideo:
        cap = cv2.VideoCapture(0)
        while cap.isOpened():
            # Grab a single frame of video
            ret, frame = cap.read()
            labels = []
            Category_Box(frame)
            cv2.imshow('Emotion Detector',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        print("Camera closed")
        cap.release()
        cv2.destroyAllWindows()
    else:
        Category_Box(exp_frame)
        cv2.namedWindow('Emotion Detector', cv2.WINDOW_NORMAL)        
        cv2.imshow('Emotion Detector',exp_frame)
        cv2.waitKey(0)
        cv2.destroyWindow('Emotion Detector')

In [48]:
def Category_Box(frame):
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)
        for (x,y,w,h) in faces:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,0),2)
                    roi_gray = gray[y:y+h,x:x+w]
                    roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
                # rect,face,image = face_detector(frame)


                    if np.sum([roi_gray])!=0:
                        roi = roi_gray.astype('float')/255.0
                        roi = img_to_array(roi)
                        roi = np.expand_dims(roi,axis=0)

                    # make a prediction on the ROI, then lookup the class

                        preds = classifier.predict(roi)[0]
                        label=class_labels[preds.argmax()]
                        label_position = (x,y)
                        cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,0),3)
                    else:
                        cv2.putText(frame,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)    

### Image Prediction:

In [28]:
imageFrame = cv2.imread('D:\Facial-CNN\TestImages\InputImages\Test1.jpg')
print('Image shape:', imageFrame.shape)  # tuple of (511, 767, 3)

# Getting spatial dimension of input image
h, w = imageFrame.shape[:2]  # Slicing from tuple only first two elements
# Check point
# Showing height an width of image
print('Image height={0} and width={1}'.format(h, w))  # 511 767
PredictResult(False,imageFrame)

Image shape: (849, 849, 3)
Image height=849 and width=849


#### Save multiple predicted images in the destination folder.

In [67]:
imageArray = ['Test1.jpg','Test2.jpg','Test3.jpg','Test4.jpg','Test5.jpg','Test6.jpg','Test7.jpg','Test8.jpg','Test9.jpg']

In [78]:
for image in imageArray:
    imageFrame1 = cv2.imread(f'D:\Facial-CNN\TestImages\InputImages\{image}')
    print('Image shape:', imageFrame1.shape)  # tuple of (511, 767, 3)

    # Getting spatial dimension of input image
    h, w = imageFrame1.shape[:2]  # Slicing from tuple only first two elements
    # Check point
    # Showing height an width of image
    print('Image height={0} and width={1}'.format(h, w))  # 511 767
    PredictResult(False,imageFrame1)
    cv2.imwrite(f"D:\Facial-CNN\TestImages\OutputImages\{image}",imageFrame1)

Image shape: (849, 849, 3)
Image height=849 and width=849
Image shape: (530, 800, 3)
Image height=530 and width=800
Image shape: (1200, 1200, 3)
Image height=1200 and width=1200
Image shape: (375, 700, 3)
Image height=375 and width=700
Image shape: (400, 400, 3)
Image height=400 and width=400
Image shape: (892, 892, 3)
Image height=892 and width=892
Image shape: (1390, 1300, 3)
Image height=1390 and width=1300
Image shape: (417, 626, 3)
Image height=417 and width=626
Image shape: (563, 800, 3)
Image height=563 and width=800


### Live Video Capture

In [27]:
PredictResult(True)

Camera closed
